In [ ]:
import warnings
import os, gc, io, requests, math, time
import numpy as np
import pandas as pd
import tensorflow as tf
import dask.dataframe as dd
import matplotlib.pyplot as plt     
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from sklearn.impute import SimpleImputer
from IPython.display import clear_output , Markdown

from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.manifold import Isomap
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.manifold import TSNE

import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.decomposition import TruncatedSVD
import statsmodels.api as sm
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

!pip install pmdarima
!pip install ivis
from ivis import Ivis
from pmdarima.preprocessing import BoxCoxEndogTransformer

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from tqdm.notebook import tqdm

%matplotlib inline
warnings.filterwarnings("ignore")
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
plt.style.use('bmh')
plt.rcParams['figure.figsize'] = [20, 5]  # width, height
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

clear_output()

# Loading Data and Preprocessing

In [ ]:
%%time
train = dd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv' ).compute().sort_values('ts_id')
features = [c for c in train.columns if "feature" in c]
non_features = [c for c in train.columns if "feature" not in c]

train['weight_x_resp'] = train['weight'] * train['resp']
train['resp_mean'] = (train['resp_1']+train['resp_2']+train['resp_3']+train['resp_4'])/4

#Getting Target Variables
train['resp_target'] = (train['resp'] > 0).astype(np.int8)
train['resp_mean_target'] = (train['resp_mean'] > 0).astype(np.int8)

In [ ]:
%%time
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype.name

        if col_type not in ['object', 'category', 'datetime64[ns, UTC]']:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df
train = reduce_mem_usage(train)

# Visualizations

As JointPlot Suggest The Resp data is standardized 

In [ ]:
Markdown('<li>As JointPoint Suggest there is no Statistical Relation Between Resp and Weight.\
          <li>Also Resp Column is <b>Pre-Standardized with Mean : {} and STD : {}</b>.\
          <li>The Data is Lighly Skewed with Skewness : {} \
          <li>Kurtosis Of Series Show its a Leptokurtic Distribution(Kurtosis > 3) With Kurtosis : {}\n'\
         .format(train['resp'].mean() , train['resp'].std() , scipy.stats.skew(train['resp'].values) ,  scipy.stats.kurtosis(train['resp'].values)))

In [ ]:
%%time
def get_stats(series , i , outlier_limit = 1.5 ) :
    describe = {}
    time_describe = {}
    describe['Column'] = i
    s_time = time.time()
    start_time = s_time
    
    describe['Mean'] = series.mean();time_describe['Mean'] = time.time() - start_time;start_time = time.time() #Mean of Series    
    describe['Median'] = series.median();time_describe['Median'] = time.time() - start_time;start_time = time.time() #Median of Series
    describe['STD'] = series.std();time_describe['STD'] = time.time() - start_time;start_time = time.time() #Standard Deviation of Series
    describe['Minimum'] = series.min();time_describe['Minimum'] = time.time() - start_time;start_time = time.time() #Minimum of Series
    describe['Maximum'] = series.max();time_describe['Maximum'] = time.time() - start_time;start_time = time.time() #Maximum of Series
    
    
    describe['Q1'] , describe['Q3'] = series.quantile([0.25 , 0.75]) # Quantile 1 and 3 of Series   
    describe['InterQuantile'] = describe['Q3'] - describe['Q1'] # Interquantile Range
    # Calculating Number of Outliers Take lot of time
#     lower_bound = describe['Q1'] - (outlier_limit * describe['InterQuantile']) 
#     upper_bound = describe['Q3'] + (outlier_limit * describe['InterQuantile'])
#     describe['Number of Outliers'] = len([x for x in series.values if x <= lower_bound or x >= upper_bound]) #Number of Outliers
    time_describe['Quantiles'] = time.time() - start_time;start_time = time.time()
    
    describe['Nulls'] = series.isnull().sum();time_describe['Nulls'] = time.time() - start_time;start_time = time.time() #Number of Null Values
    describe['Skewness'] = scipy.stats.skew(series.values);time_describe['Skewness'] = time.time() - start_time;start_time = time.time() # Skewness of Series
    describe['Kurtosis'] = scipy.stats.kurtosis(series.values);time_describe['Kurtosis'] = time.time() - start_time;start_time = time.time() #Kurtosis (Peakness in Distributed Data)
    describe['Variation'] = scipy.stats.skew(series.values);time_describe['Variation'] = time.time() - start_time;start_time = time.time() #Ratio of the biased standard deviation to the mean.
    describe['Entroy'] = scipy.stats.entropy(series.values);time_describe['Entroy'] = time.time() - start_time;start_time = time.time() #Entropy of Series
    
    describe['Total_Time'] = time.time() - s_time
    return [describe , time_describe]

def Custom_Describe(data , columns , sample = 100000):
    r_data = []
    t_data = []
    sample_data = data.sample(sample)
    for i in tqdm(columns):
        return_data = get_stats(sample_data[i] , i)
        r_data.append(return_data[0])
        t_data.append(return_data[1])
    r_data = pd.DataFrame(r_data)
    t_data = pd.DataFrame(t_data)
    return r_data , t_data

r_data  , t_data = Custom_Describe(train , non_features[1:-1]+features + ['resp_mean','weight_x_resp'] )
r_data.index = r_data.Column
r_data.drop(columns = ['Column'] , inplace = True)
r_data.to_csv('Train_Details.csv')
display(r_data.head(10))

In [ ]:
r_data.loc[[x  for x in r_data.index.values if 'resp' in x]][['InterQuantile' , 'Skewness' , 'Kurtosis' , 'Variation' , 'Mean' , 'STD']].sort_values(['InterQuantile'])

In [ ]:
print('Percentage of Weight is equal to 0 is ',train[train['weight'] == 0.0].shape[0]*100/train.shape[0])

In [ ]:
%%time
sns.jointplot(x = 'resp' ,y= 'weight' , data = train , kind = 'scatter')

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 6))
sns.distplot(train['resp'], ax=axs[0])
sns.distplot(train['weight'], ax=axs[1])

In [ ]:
correlations = train.sample(100000).corr(method='pearson')
fig, axs = plt.subplots(figsize=(16, 16))
sns.heatmap(correlations)

The Cummulative Product of Resp Show Increment which Show the Profits are Showing Up with Days.

In [ ]:
(1+train.groupby('date')['resp'].mean()).plot(label = 'Resp Mean')
(1+train.groupby('date')['weight_x_resp'].mean()).cumprod().plot(label = 'Cummulative Weight*Resp')
(1+train.groupby('date')['resp'].mean()).cumprod().plot(label = 'Cummulative Resp')
(1+train.groupby('date')['resp'].std()).plot(label = 'Resp Standard Deviation')
plt.legend()

In [ ]:
(1+train.groupby('date')['weight_x_resp'].mean()).plot(label = 'Mean Weight*Resp')
(1+train.groupby('date')['resp'].mean()).plot(label = 'Resp Mean')
plt.legend()

In [ ]:
(1+train.groupby('date')['resp'].mean()).cumprod().plot()
(1+train.groupby('date')['weight_x_resp'].mean()).cumprod().plot()
(1+train.groupby('date')['resp_mean'].mean()).cumprod().plot()
plt.legend()

In [ ]:
(1+train.groupby('date')['resp_1'].mean()).cumprod().plot()
(1+train.groupby('date')['resp_2'].mean()).cumprod().plot()
(1+train.groupby('date')['resp_3'].mean()).cumprod().plot()
(1+train.groupby('date')['resp_4'].mean()).cumprod().plot()
plt.legend()

In [ ]:
Season_Period = 7
Model_Behaviour = "add"

data = train.groupby('date')['resp'].mean()

if Model_Behaviour == 'mul':
    data = data + 1
seas_d=sm.tsa.seasonal_decompose(data ,model=Model_Behaviour,period=Season_Period , extrapolate_trend='freq')
clear_output()

Plt_Col = 2
Plt_Row = 1
plt.figure(figsize=(20,16/Plt_Col))

plt.subplot(Plt_Row,Plt_Col , 1)
plt.plot(seas_d.seasonal.values , label = 'Estimated Seasonal Component')
# plt.fill_between(list(range(seas_d.nobs[0])) ,0, seas_d.seasonal.values , alpha =0.4)
plt.legend()

plt.subplot(Plt_Row,Plt_Col , 2)
plt.plot(seas_d.resid.values , label = 'Estimated Residual Component')
plt.fill_between(list(range(seas_d.nobs[0])) ,0, seas_d.seasonal.values , alpha =0.4 , label = 'Estimated Seasonal Components')
plt.legend()

plt.plot()

plt.figure(figsize=(20,16/Plt_Col))
plt.plot(seas_d.observed.values , label = 'Observed Component')
plt.plot(seas_d.trend.values , label = 'Estimated Trend Component')
plt.legend()
plt.plot()


In [ ]:
data = 1 + train.groupby('date')['weight'].mean()
Plt_Col = 4
freq = []

freq.append([data.values.reshape(-1)  , 'Original Data'])
freq.append([data.diff().values.reshape(-1)  , 'Difference of Original Data'])
# freq.append([data.cumprod().values.reshape(-1)  , 'Cummulative Product Plot'])
freq.append([data.cumsum().values.reshape(-1)  , 'Cummulative Sum Plot'])
freq.append([np.log1p(data).values.reshape(-1) , 'Log1p Fuction'])
freq.append([np.sqrt(data).values.reshape(-1) , 'Square Root Function'])
freq.append([np.cbrt(data).values.reshape(-1) , 'Cube Root Function'])
freq.append([np.sqrt(np.sqrt(data)).values.reshape(-1) , 'Quad Root Function'])
freq.append([np.square(data).values.reshape(-1) , 'Square Function'])
freq.append([preprocessing.MinMaxScaler().fit_transform(data.values.reshape(-1,1)).reshape(-1) , 'Min Max Scaler Function'])
freq.append([preprocessing.RobustScaler().fit_transform(data.values.reshape(-1,1)).reshape(-1) , 'Robust Scaler Function'])
freq.append([preprocessing.StandardScaler().fit_transform(data.values.reshape(-1,1)).reshape(-1) , 'Standard Scaler Function'])
freq.append([preprocessing.PowerTransformer().fit_transform(data.values.reshape(-1,1)).reshape(-1) , 'Power Transformer Scaler'])
freq.append([sm.tsa.filters.bkfilter(data).values.reshape(-1) , 'BKFilter Band Pass Filter'])
freq.append([sm.tsa.filters.hpfilter(data)[1].values.reshape(-1) , 'HPFilter Smoothing Filter'])
freq.append([sm.tsa.filters.hpfilter(data.cumprod())[1].values.reshape(-1) , 'HPFilter  on CumPord'])
freq.append([BoxCoxEndogTransformer().fit_transform(data+0.00000001)[0], 'BoxCoxEndogTransformer Filter'])

Plt_Row = math.ceil(len(freq)/Plt_Col)
plt.figure(figsize=(16,12*Plt_Row/Plt_Col))

for i,j in enumerate(freq):
    plt.subplot(Plt_Row , Plt_Col , i+1)
    plt.plot(j[0], label = j[1] , linewidth = .8)
  
    plt.legend(loc = 2)

# About Missing Values

In [ ]:
null_percent = []
for i in train.columns:
    null_percent.append([i , train[i].isnull().sum()])
null_percent = pd.DataFrame(null_percent , columns = ['Column', 'Missings'])
null_percent['P_Missings'] = (null_percent['Missings']/train.shape[0])*100
null_percent.sort_values(by =['Missings'], ascending = False, inplace = True)
fig, axs = plt.subplots(1, 1, figsize=(16, 6))
null_percent.reset_index(drop = True)['P_Missings'].plot(ax = axs)
plt.vlines(x = len(null_percent[null_percent['Missings'] != 0]), ymin = 0, ymax = null_percent['P_Missings'].max(),
           colors = 'purple', 
           label = 'Zero Values') 
  
plt.xlabel('Columns')
plt.ylabel('Percentage of Missing %')
plt.legend()
null_percent.index = null_percent.Column
null_percent.drop(columns = ['Column'] , inplace = True)

In [ ]:
null_percent.sort_index().loc[features]['Missings'].plot()

In [ ]:
null_percent.head(10)

## Understanding Missing Values for Top Missed Columns

In [ ]:
day = 54
(train[train['date'] == day]['resp']*100).plot()
(train[train['date'] == day]['feature_28']+40).plot()
(train[train['date'] == day]['feature_27']+20).plot()
(train[train['date'] == day]['feature_18']+60).plot()
(train[train['date'] == day]['feature_7']+80).plot()
(train[train['date'] == day]['feature_8']+100).plot()
plt.legend()

## Understanding consecutive Days

In [ ]:
(train[train['date'] == 0]['feature_28'].reset_index(drop = True)+0 ).plot(label = 'Day 0')
(train[train['date'] == 1]['feature_28'].reset_index(drop = True)+10).plot(label = 'Day 1')
(train[train['date'] == 2]['feature_28'].reset_index(drop = True)+10).plot(label = 'Day 2')
(train[train['date'] == 3]['feature_28'].reset_index(drop = True)+20).plot(label = 'Day 3')
(train[train['date'] == 4]['feature_28'].reset_index(drop = True)+30).plot(label = 'Day 4')
(train[train['date'] == 5]['feature_28'].reset_index(drop = True)+40).plot(label = 'Day 5')
(train[train['date'] == 6]['feature_28'].reset_index(drop = True)+50).plot(label = 'Day 6')
(train[train['date'] == 7]['feature_28'].reset_index(drop = True)+60).plot(label = 'Day 7')
(train[train['date'] == 8]['feature_28'].reset_index(drop = True)+70).plot(label = 'Day 8')
plt.legend()

In [ ]:
### Highest count Days
(train[train['date'] == 44]['feature_28'].reset_index(drop = True)+0 ).plot(label = 'Day 44')
(train[train['date'] == 45]['feature_28'].reset_index(drop = True)+20).plot(label = 'Day 45')
(train[train['date'] == 459]['feature_28'].reset_index(drop = True)+40).plot(label = 'Day 459')
(train[train['date'] == 38]['feature_28'].reset_index(drop = True)+60).plot(label = 'Day 38')
(train[train['date'] == 85]['feature_28'].reset_index(drop = True)+80).plot(label = 'Day 85')
plt.legend()

In [ ]:
### Least Count Days
(train[train['date'] == 2]['feature_28'].reset_index(drop = True)).plot(label = 'Day 2')
(train[train['date'] == 36]['feature_28'].reset_index(drop = True)+20).plot(label = 'Day 36')
(train[train['date'] == 294]['feature_28'].reset_index(drop = True)+10).plot(label = 'Day 294')
(train[train['date'] == 270]['feature_28'].reset_index(drop = True)+30).plot(label = 'Day 270')
plt.legend()

* There is Missing Part every Day at beginning and Middle.
* That must be The Gap of Some Sort that Inflences the Trade Happening (Except of Day  2 & 294)

# Count of Each Day

In [ ]:
count_data = train.groupby(['date'])['resp'].count()
plt.axhline(y = count_data.mean() , linestyle = 'dashed' , label = 'Count Mean', color = 'm' , linewidth = 1.5)
plt.axhline(y = count_data.median() , linestyle = 'dashed' , label = 'Count Median', color = 'c' , linewidth = 1)
sm.tsa.filters.hpfilter(count_data , lamb = 50)[1].plot(label = 'Count Per Day (Filtered)' , color = '#0072B2',  linewidth = 1.3)
count_data.plot(label = 'Counts Per Day' , linewidth = 0.85 , color = '#0072B2')
plt.legend()